In [676]:
import numpy as np
import cv2
from PIL import Image
import imutils
import pytesseract

img = cv2.imread('IMG_0610.JPG')
h, w, c = img.shape

In [677]:
origin = img.copy()

In [678]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(gray, 100, 250)

In [679]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

In [680]:
def four_point_transform(image, pts):
    rect = order_points(pts)
    tl, tr, br, bl = rect
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype = "float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

In [681]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
cnts = cv2.findContours(closed.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)
    if len(approx == 4):
        screenCnt = approx
        break
cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 2)
screenCnt

array([[[1114,  156]],

       [[ 384,  934]],

       [[1705, 2101]],

       [[2386, 1280]]], dtype=int32)

In [682]:
warped = four_point_transform(origin, screenCnt.reshape(4, 2))
card_height, card_width, channels = warped.shape

In [683]:
warped = cv2.resize(warped, (int(card_width*500/card_height), 500))
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
#warped = cv2.bilateralFilter(warped, 9, 75, 75)
cv2.imwrite('Cardsv5.jpg', warped)

True

In [686]:
type(Image.open('Cardsv5.jpg'))

PIL.JpegImagePlugin.JpegImageFile

In [674]:
cv2.imshow('Origin', origin)
cv2.imshow('Edged', warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [684]:
print (pytesseract.image_to_string(Image.open('Cardsv5.jpg'), lang='vie'))

J HỌC VIỆN
PL CÔNG N...'iỆ BƯU CHÍNH VIỄN THÔNG

2 ˆ-
THE SINH VIÊN
Họ và tên: Đỗ Việt Anh
Sinh ngày: 03/01/1999
Hộ khẩu TT: Q. Tây;Hồ -'tlà Nội
Lớp: D17CQỆN09-Bế ` Hệ: ĐHCQ
Ngành: CNTT-> Khóa: 2017-2022

Mã SV
s4zpcewoo9 ... | ÏÏÏlÏllÏÏ|ll[llll[IlllillfillllllIllllll[Illlll


In [566]:
im.info

{'jfif': 257, 'jfif_version': (1, 1), 'jfif_unit': 0, 'jfif_density': (1, 1)}